In [5]:
import sys
import json
import numpy as np

sys.path.append('../')
from genetic_circuit_scoring import CircuitMapping

let's load the data to develop a solution

In [43]:
path = '../genetic_logic_synthesis/genetic_circuit_scoring/example/'

with open(path + 'genetic_gate_library.json') as library_file:
    library_data = json.load(library_file)
with open(path + 'majority_mapping.json') as majority_mapping_file:
    majority_mapping_data = json.load(majority_mapping_file)
with open(path + 'majority_tuning.json') as majority_tuning_file:
    majority_tuning_data = json.load(majority_tuning_file)

In [44]:
circuit_mapping = CircuitMapping(library_data)
circuit_mapping.map(majority_mapping_data)

In [45]:
circuit_mapping.score()

0.5191361249786275

In [46]:
circuit_mapping.tune(majority_tuning_data)

In [47]:
circuit_mapping.score()

0.5209209930735266

ok, so now let's havea a look at what we need to create

In [48]:
majority_tuning_data

{'gates': [{'id': 'n0', 'promoter': 1.5},
  {'id': 'n4', 'rbs': 2},
  {'id': 'n8', 'promoter': 0.5, 'rbs': 0.5}]}

In [49]:
majority_mapping_data['gates']

[{'id': 'n0', 'type': 'NOT', 'mapping': 'A1_AmtR'},
 {'id': 'n1', 'type': 'OR'},
 {'id': 'n2', 'type': 'NOT', 'mapping': 'B1_BM3R1'},
 {'id': 'n3', 'type': 'NOT', 'mapping': 'E1_BetI'},
 {'id': 'n4', 'type': 'NOT', 'mapping': 'F1_AmeR'},
 {'id': 'n5', 'type': 'OR'},
 {'id': 'n6', 'type': 'NOT', 'mapping': 'H1_HlyIIR'},
 {'id': 'n7', 'type': 'OR'},
 {'id': 'n8', 'type': 'NOT', 'mapping': 'I1_IcaRA'},
 {'id': 'n9', 'type': 'OR'}]

let's add an entry

In [50]:
new_entry = {'id': 'n6', 'promoter':0.5, 'rbs': 0.8}

In [53]:
majority_tuning_data['gates'].append(new_entry)

In [54]:
majority_tuning_data

{'gates': [{'id': 'n0', 'promoter': 1.5},
  {'id': 'n4', 'rbs': 2},
  {'id': 'n8', 'promoter': 0.5, 'rbs': 0.5},
  {'id': 'n6', 'promoter': 0.5, 'rbs': 0.8}]}

In [55]:
circuit_mapping.tune(majority_tuning_data)

In [56]:
circuit_mapping.score()

0.5221539025179964

#### Let's first create a all element that can be tuned for a given circuit

In [175]:
tuning_data = []
for dict_ in majority_mapping_data['gates']:
    if dict_['type'] == 'NOT':
        tuning_data.append({'id': dict_['id'], 'promoter':1.0, 'rbs': 1.0})

In [176]:
tuning_data

[{'id': 'n0', 'promoter': 1.0, 'rbs': 1.0},
 {'id': 'n2', 'promoter': 1.0, 'rbs': 1.0},
 {'id': 'n3', 'promoter': 1.0, 'rbs': 1.0},
 {'id': 'n4', 'promoter': 1.0, 'rbs': 1.0},
 {'id': 'n6', 'promoter': 1.0, 'rbs': 1.0},
 {'id': 'n8', 'promoter': 1.0, 'rbs': 1.0}]

In [62]:
tuning_data_sol = {'gates': tuning_data}

In [63]:
circuit_mapping.tune(tuning_data_sol)

In [64]:
circuit_mapping.score()

0.5191361249786275

#### ok, now let's define the output we need from the neural net
- we will then either multiply or add the output of the neural net to the tuning data where all values are 1

In [79]:
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Sequential, Model

In [68]:
from evostra import EvolutionStrategy

In [200]:
# create the neural net
class MODEL():
    def __init__(self, out_dim):        

        # dict to map the circuit element to char and int
        # those mapping are needed for the neural net
        self.out_dim = out_dim
        self.model = self.build_model()
        
    def build_model(self):
        """
        function to create the neural net
        """
        model = Sequential()
        model.add(Dense(256))
        model.add(BatchNormalization())
        model.add(Dense(self.out_dim))
        
        # we will first try to add, so no activation function

        return model

In [201]:
out_dim = len(tuning_data)*2
out_dim

12

let's create our model, which will take as input a dummy matrix to get a list of output

In [207]:
model = MODEL(out_dim)

In [208]:
net = model.build_model()

In [209]:
X = np.ones([5,5])

In [210]:
solution = net.predict(X)[0]

In [211]:
solution

array([ 0.03161672,  0.23497109,  0.44255808, -0.1331769 ,  0.10330744,
        0.08331902,  0.12131631,  0.47361755, -0.16213001,  0.3228344 ,
        0.12106256,  0.19425496], dtype=float32)

we will use those element to modify the dictionnary of values for promoters and rbs

let's try to fill the dict with the solution from the net

In [177]:
idx = 0
for dict_ in tuning_data:
    dict_['promoter'] += solution[idx]
    dict_['rbs'] += solution[idx+1]
    idx+=2

In [178]:
tuning_data

[{'id': 'n0', 'promoter': 1.0443906970322132, 'rbs': 1.039889894425869},
 {'id': 'n2', 'promoter': 1.3792296350002289, 'rbs': 1.0551776215434074},
 {'id': 'n3', 'promoter': 1.0, 'rbs': 1.097153514623642},
 {'id': 'n4', 'promoter': 1.0, 'rbs': 1.0},
 {'id': 'n6', 'promoter': 1.1414542496204376, 'rbs': 1.0},
 {'id': 'n8', 'promoter': 1.0, 'rbs': 1.5824653506278992}]

In [179]:
tuning_data_solution = {'gates': tuning_data}

In [180]:
circuit_mapping.tune(tuning_data_solution)

In [181]:
circuit_mapping.score()

0.5114687426644776

#### let's get the function for ES

In [212]:
def get_score(n_samples, weights):
    """
    function to get the score of a given seqence
    of elements given by the algo
    params – n_samples: number of samples sampled
    by the algo for this run
    params - weights: weights of the neural net
    """
    all_scores = []
    for i in range(n_samples):
        
        #load the current weights and sample solution
        net.model.set_weights(weights)
        solution = net.predict(X)[0]
        print('solution: ', solution)
        
        path_json = '../../genetic_logic_synthesis/genetic_circuit_scoring/example/majority_mapping.json'
        score = get_score_mapping(solution, path_json)
        
    # return the maximum of all the samples
    # because we aim for the max score
    # note: we could have optimise for the mean
    # of all our samples of a given epoch e.g.
    return np.mean(all_scores)

def get_reward(weights):
    """
    function needed by evostra (ES library)
    to optomise the weights of a neural net
    given a number to optimise, i.e. the reward
    params - weights: parameters of the neural nets,
    i.e. its weights
    """
    # n_samples is the number of samples 
    # sampled at each epoch
    n_samples = 1
    return get_score(n_samples, weights)

def get_score_mapping(solution, path):
    """
    function to get the score back from generated
    outpout of the neural net.
    params - mapping_from_net: output from the neural net
    params - path: path to the json file
    """
        
    # we load the original file
    # at every call to avoid
    # messing up with cloning
    # of nested objects
    with open(path) as file:
        list_data = json.load(file)
        
    with open('../../genetic_logic_synthesis/genetic_circuit_scoring/example/genetic_gate_library.json') as library_file:
        library_data = json.load(library_file)

    circuit_mapping = CircuitMapping(library_data)
    circuit_mapping.map(list_data)
    
    # we create the tuning data with a factor of 1
    tuning_data = []
    for dict_ in list_data['gates']:
        if dict_['type'] == 'NOT':
            tuning_data.append({'id': dict_['id'], 'promoter':1.0, 'rbs': 1.0})
    
    # now we will this dict
    # with the net output
    idx = 0
    for dict_ in tuning_data:
        dict_['promoter'] += solution[idx]
        dict_['rbs'] += solution[idx+1]
        idx+=2
        
    tuning_data_sol = {'gates': tuning_data}  
    print('tuning_data_sol ', tuning_data_sol)
    circuit_mapping.tune(tuning_data_sol)
    print('score: ', circuit_mapping.score())
    
    return circuit_mapping.score()

let's finally run the algo

In [213]:
# run the evolution strategies (ES)
# note: if you want, you can play with the paramters.
# those seems to give reasonable results
es = EvolutionStrategy(net.model.get_weights(), 
                       get_reward, 
                       population_size=2, 
                       sigma=0.01,  # noise std deviation
                       learning_rate=0.001, 
                       decay=0.995,
                       num_threads=1)

es.run(2)

solution:  [ 0.07430971  0.24565123  0.5065814  -0.15450804  0.04086089 -0.02447774
  0.11658062  0.6242171  -0.18410915  0.278344    0.13235936  0.30078766]
tuning_data_sol  {'gates': [{'id': 'n0', 'promoter': 1.0743097141385078, 'rbs': 1.2456512302160263}, {'id': 'n2', 'promoter': 1.506581425666809, 'rbs': 0.845491960644722}, {'id': 'n3', 'promoter': 1.040860891342163, 'rbs': 0.9755222592502832}, {'id': 'n4', 'promoter': 1.1165806204080582, 'rbs': 1.6242170929908752}, {'id': 'n6', 'promoter': 0.8158908486366272, 'rbs': 1.2783440053462982}, {'id': 'n8', 'promoter': 1.132359355688095, 'rbs': 1.3007876574993134}]}
score:  0.5203859533260793
solution:  [-0.0413533   0.22107005  0.4754806  -0.14393836  0.10090262  0.1231711
  0.17323868  0.3538392  -0.14608994  0.2983724   0.09423239  0.1960909 ]
tuning_data_sol  {'gates': [{'id': 'n0', 'promoter': 0.9586467035114765, 'rbs': 1.2210700511932373}, {'id': 'n2', 'promoter': 1.4754805862903595, 'rbs': 0.8560616374015808}, {'id': 'n3', 'promote

/Library/Frameworks/Python.framework/Versions/2.7/envs/bda/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '
/Library/Frameworks/Python.framework/Versions/2.7/envs/bda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Library/Frameworks/Python.framework/Versions/2.7/envs/bda/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ok, the net output nan value. I guess the evolution strategies are messing a bit too much with its weights for now. With more tuning, this should probably give a working solution.